In [1]:
from __future__ import print_function
from __future__ import division

import os
import time
import curses
import numpy as np
import cv2
import src.viztools as vv
import src.utils as util

 ### TODO
 * refactor key input handling across classes (change phase keys in Alien)
 * border zoom out black masking

In [ ]:
"""vid-viz main loop

KEYBOARD INPUTS:
    number keys - choose effect (see viztools.py for detailed keyboard input for each effect)
        0 - thresholding
        1 - alien
        2 - rgbwalk
        3 - rgbburst
        4 - huebloom
        5 - hueswirl
        6 - huecrusher
    q - quit effect (then number keys are once again available for choosing a new effect)
    ` - enable toggling of border effects ('t' to toggle, tab to switch processing/border order)
    spacebar - cycle through sources
    backspace - quit border effect editing
    esc - exit loop
"""

# 'cam' | 'video' | 'image' | 'gen'
SOURCE_INDEX = 0
SOURCE_LIST = []
SOURCE_LIST.append({
    'file_loc': None,
    'file_type': 'cam'})
SOURCE_LIST.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/snowflake-02.mp4',
    'file_type': 'video'})
SOURCE_LIST.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/tree-00.jpg',
    'file_type': 'image'})
SOURCE_LIST.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/honeycomb-00.jpg',
    'file_type': 'image'})
# SOURCE_LIST.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-00.jpg',
#     'file_type': 'image'})
# SOURCE_LIST.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-01.jpg',
#     'file_type': 'image'})
# SOURCE_LIST.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-02.jpg',
#     'file_type': 'image'})
# SOURCE_LIST.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-03.jpg',
#     'file_type': 'image'})
SOURCE_LIST.append({
    'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-04.jpg',
    'file_type': 'image'})
# SOURCE_LIST.append({
#     'file_loc': '/media/data/Dropbox/Git/vid-viz/data/waves-05.jpg',
#     'file_type': 'image'})

SOURCE = None
NUM_SOURCES = len(SOURCE_LIST)

# general user parameters
DISP_FULL_SCREEN = False
# FRAME_WIDTH = int(1024)
# FRAME_HEIGHT = int(768)
FRAME_WIDTH = int(1980/2)
FRAME_HEIGHT = int(1200/2)
TARGET_FPS = 20.0

# initialize key handler
effect_index = None
key_list = [False for i in range(256)]

# set pre/post-processing options
post_process = False      # whether or not post-processing is active effect
post_process_pre = False  # whether post-processing pre/proceeds other effects

# initialize processing objects
effects = [
    vv.Threshold(),      # 0
    vv.Alien(),          # 1
    vv.RGBWalk(),        # 2
    vv.RGBBurst(),       # 3
    vv.HueBloom(),       # 4
    vv.HueSwirl(),       # 5
    vv.HueSwirlMover(),  # 6
    vv.HueCrusher()]     # 7
num_effects = len(effects)
border = vv.Border()
postproc = vv.PostProcess()
    
new_source = True
while(True):

    time_pre = time.time()
    
    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    key_list[key] = True
    if key == 27:
        # escape key; exit program
        break
    elif key == ord('q'):
        # quit current effect
        effect_index = None
    elif key == ord('\b'):
        post_process = False
    elif key == ord('`'):
        post_process = True
    elif key == ord(' '):
        SOURCE_INDEX = (SOURCE_INDEX + 1) % NUM_SOURCES
        new_source = True
    elif key == ord('\t'):
        # only change post-processing order if in post-process mode
        if post_process:
            post_process_pre = not post_process_pre
            
    if new_source:
        # reset necessary parameters
        new_source = False
        effect_index = None
        fr_count = 0
        for _, effect in enumerate(effects):
            effect.reset()
        border.reset()
        postproc.reset()
        # load source
        if SOURCE is 'cam' or SOURCE is 'video':
            cap.release()
        SOURCE = SOURCE_LIST[SOURCE_INDEX]['file_type']
        if SOURCE is 'cam':
            cap = cv2.VideoCapture(0)
            TOTAL_FRAME_COUNT = float('inf')
            frame_mask = None
        elif SOURCE is 'video':
            cap = cv2.VideoCapture(SOURCE_LIST[SOURCE_INDEX]['file_loc'])
            # TARGET_FPS = cap.get(cv2.CAP_PROP_FPS)
            TOTAL_FRAME_COUNT = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_mask = None
        elif SOURCE is 'image':
            frame_orig = cv2.imread(SOURCE_LIST[SOURCE_INDEX]['file_loc'])
            TOTAL_FRAME_COUNT = float('inf')
            frame_mask = np.copy(frame_orig)   
        elif SOURCE is 'gen':
            pass
        else:
            print('Invalid SOURCE type')          
    
    # get frame and relevant info
    if SOURCE is 'cam' or SOURCE is 'video':
        ret, frame = cap.read()
    elif SOURCE is 'image':
        frame = np.copy(frame_orig)
    elif SOURCE is 'gen':
        frame = None
        
    # get uniform frame sizes 
    frame = util.resize(frame, FRAME_WIDTH, FRAME_HEIGHT)
    
    # update current effect
    if effect_index is None:
        for num in range(10):
            if key == ord(str(num)):
                effect_index = num
                key_list[key] = False
    
    # apply borders before effect
    if post_process_pre:
        if post_process:
            frame = border.process(frame, key_list)
        else:
            frame = border.process(frame, key_list, key_lock=True)
    
    # process frame
    if effect_index is not None and effect_index < num_effects:
        if post_process:
            frame = effects[effect_index].process(frame, key_list, key_lock=True)            
        else:
            frame = effects[effect_index].process(frame, key_list)
            
    # apply borders after effect
    if not post_process_pre:
        if post_process:
            frame = border.process(frame, key_list)
#             frame = postproc.process(frame, key_list)
        else:
            frame = border.process(frame, key_list, key_lock=True)
#             frame = postproc.process(frame, key_list, key_lock=True)
    
    # display frame
    if DISP_FULL_SCREEN:
        cv2.namedWindow('frame', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('frame', frame)
    else:
        cv2.imshow('frame', frame)
        if fr_count == 0:
            cv2.moveWindow('frame', 0, 0)
    
    # control animation
    fr_count += 1
    if fr_count == TOTAL_FRAME_COUNT:
        # reset frame postion to 1 (not zero so window isn't moved)
        cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
        fr_count = 1

    # calculate, limit and output fps
    time_tot = time.time() - time_pre
    if time_tot < 1/TARGET_FPS:
        time.sleep(1/TARGET_FPS - time_tot)
    time_tot = time.time() - time_pre
    print('\r%03i fps' % (1.0 / time_tot), end='')
#     print('\r%02i' % effects[5].PROPS[4]['VAL'], end='')
#     stdscr.addstr(0, 0, '%03i fps' % (1.0 / time_tot))
#     stdscr.addstr(1, 0, '%05i frames' % fr_count)
#     stdscr.refresh()
    
#     frame_final = np.copy(frame)
    
    # reset key list (for pressed keys that are not reset by effect object(s))
    key_list[key] = False
    
if SOURCE is 'cam' or SOURCE is 'video':
    cap.release()
cv2.destroyWindow('frame')

007 fps

In [40]:
reload(vv)
reload(util)

<module 'src.utils' from 'src/utils.pyc'>

In [ ]:
INF = 100000
a= 5
a = a % INF
print(a)
print(type(a))

In [ ]:
%timeit cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

In [ ]:
import PIL.Image
PIL.Image.fromarray(frame_final).save('/media/data/Dropbox/image_play/hue-bloom/final/tree-00.jpg', 'jpeg')

In [ ]:
img1 = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test_img.png')
img1 = util.resize(img1, 1920, 1200)

# img0 = 0
# img2 = cv2.addWeighted(img0, 0.1, img1, 0.9, 0)
# %timeit cv2.addWeighted(img0, 0.1, img1, 0.9, 0)

# im_width, im_height, _ = img1.shape
# %timeit cv2.warpAffine(img1, \
#             np.float32([[1, 0, 100], \
#                         [0, 1, 200]]), \
#             (im_width, im_height))

%timeit cv2.threshold(img1, 128, 255, cv2.THRESH_TOZERO)

In [ ]:
frame = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/test_img.png')
frame = util.resize(frame, int(1920/2), int(1200/2))

# extract hue values
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
[im_height, im_width, _] = frame.shape
hue_final = np.zeros((im_height, im_width), dtype='uint8')

chunk_center = 128
chunk_widths_og = 128
chunk_width = 8
center_offset = 0

# human-readable names
num_chunks = 3
center_offset = 0
chunk_width = 8

chunk_widths_og = int(256 / num_chunks)
chunk_range_mins = np.zeros((num_chunks, 1), 'uint8')
chunk_range_maxs = np.zeros((num_chunks, 1), 'uint8')
chunk_centers = np.zeros((num_chunks, 1), 'uint8')
for chunk in range(num_chunks):
    chunk_range_mins[chunk] = chunk * chunk_widths_og
    chunk_range_maxs[chunk] = (chunk + 1) * chunk_widths_og - 1
    chunk_centers[chunk] = chunk_range_mins[chunk] + int(chunk_widths_og / 2)
chunk_range_maxs[-1] = 255
    
# process image
if len(frame.shape) == 3:
    [im_height, im_width, _] = frame.shape
elif len(frame.shape) == 2:
    [im_height, im_width] = frame.shape

# extract hue values
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
hue_final = np.zeros((im_height, im_width), dtype='uint8')

# scale whole image
scale = chunk_width / chunk_widths_og
frame_scaled = cv2.addWeighted(0, 1 - scale, frame, scale, 0)

for chunk in range(num_chunks):

    chunk_center = chunk_centers[chunk]

    # get rid of values outside bounds of chunk
    hue_mask = cv2.inRange(frame[:, :, 0],
                           chunk_range_mins[chunk],
                           chunk_range_maxs[chunk])
    # hue_mask is now a mask for the location of values in a specific
    # hue band;
    hue = frame_scaled[:, :, 0] + chunk_center - int(chunk_width / 2) + \
        center_offset
    hue = cv2.bitwise_and(hue, hue_mask)

    # put into final hue array (bitwise or takes all nonzero vals)
    hue_final = cv2.bitwise_or(hue_final, hue)

# return to bgr format
frame[:, :, 0] = hue_final
frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)

while(True):
    
    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    print('\r%g' % key, end='')
    
    if key == 27:
        # escape key
        break
        
    cv2.imshow('frame', frame)
        
cv2.destroyWindow('frame')

In [ ]:
img_orig = cv2.imread('/media/data/Dropbox/Git/vid-viz/data/honeycomb_01.jpg')
img_orig = cv2.resize(
    img_orig,
    None,
    fx=0.5,
    fy=0.5,
    interpolation=cv2.INTER_LINEAR)

# get mask
img_gray = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
img_gray = cv2.medianBlur(img_gray, 11)
img_mask = cv2.adaptiveThreshold(
    img_gray,
    255, # thresh ceil
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY_INV,
    21, # thresh block
    4 # thresh bias
)
img_mask2 = cv2.adaptiveThreshold(
    img_gray,
    255, # thresh ceil
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY,
    21, # thresh block
    4 # thresh bias
)
[mask_height, mask_width] = img_mask.shape

# get hue background
back_height = 10 #int(mask_height/60.0)
back_width = 10 #int(mask_width/60.0)
img_back = np.ones((back_height, back_width, 3))
img_back[:, :, 0] = np.random.rand(back_height, back_width)
img_back = cv2.resize(img_back,
    (mask_width, mask_height),
    interpolation=cv2.INTER_CUBIC)
img_back = 255.0 * img_back
img_back = img_back.astype('uint8')
img_back = cv2.cvtColor(img_back, cv2.COLOR_HSV2BGR)

# get blurred background
img_back2 = img_back
for chan in range(3):
    img_back2[:, :, chan] = cv2.bitwise_and(img_back[:, :, chan], img_mask)
REDUCE_SIZE = False
if REDUCE_SIZE:
    img_back2 = cv2.resize(
        img_back2,
        None,
        fx=0.10,
        fy=0.10,
        interpolation=cv2.INTER_LINEAR)
    img_back2 = cv2.GaussianBlur(img_back2, (5,5), 0)
    img_back2 = cv2.resize(
        img_back2,
        (mask_width, mask_height),
        interpolation=cv2.INTER_LINEAR)
else:
    img_back2 = cv2.GaussianBlur(img_back2, (71, 71), 0)

# remask blurred background
img = img_back2
for chan in range(3):
    img[:, :, chan] = cv2.bitwise_and(img_back2[:, :, chan], img_mask2)
    
while(True):
    
    # get keyboard input
    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        # escape key
        break
        
    cv2.imshow('frame', img_back2)
    cv2.moveWindow('frame', 0, 0)

cv2.destroyWindow('frame')

In [ ]:
%whos

### To patrol-cycle an mp4 with ffmpeg
* ffmpeg -i source.mp4 -vf reverse source_rev.mp4
* ffmpeg -f concat -i concat_list.txt -c copy concated.mp4

##### concat_list.txt
file '/path/to/vid0.mp4'

file '/path/to/vid1.mp4'